In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings

import pandas as pd
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)   

# Scraping + Basic Data Prep


* concat all 26 websites in 1 dataframe
* adjust dataframe
* change datatype

In [6]:
main_url = 'https://sgulyano.github.io/eval/'

html_url = ['02226005.html',
            '08846113.html',
            '19689783.html',
            '24363682.html',
            '24674969.html',
            '26742703.html',
            '33472214.html',
            '33561138.html',
            '33565025.html',
            '41557226.html',
            '43951333.html',
            '44868136.html',
            '45435188.html',
            '48033334.html',
            '51435303.html',
            '53231654.html',
            '60189297.html',
            '61116032.html',
            '66887754.html',
            '68346670.html',
            '71728345.html',
            '79550445.html',
            '83255004.html',
            '94026030.html',
            '95860997.html',
            '96905621.html']
eval_url = [main_url + url for url in html_url]

def get_table_data(item):
    data = []
    table_body = item.find("tbody")
    rows = table_body.find_all("tr")
    for row in rows :
        cols = row.find_all(["td","th"])
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])
    return data
    
dfs = []

for j in range(26):
    page  = j
    r = requests.get(eval_url[page])
    soup = BeautifulSoup(r.content,"html.parser")
    tables = soup.find_all("table")
    head = pd.DataFrame(get_table_data(tables[0]))
    section = head.iloc[1,1]

# Adjust dataframe
    df = pd.DataFrame(get_table_data(tables[2]))
    df.dropna(how="all",inplace=True)
    df.columns = df.iloc[1,:]
    rows = [0,1]
    df.drop(rows,axis=0,inplace=True)
    df.drop(df[df.ที่.str.contains("รวม")].index , inplace= True)


    df["topic"] = ""
    ind = 0
    for i in df["ที่"]:
        if len(i) > 1:
            topic = i
            df.iloc[ind,16] = topic
            ind += 1
        else:
            df.iloc[ind,16] = topic
            ind +=1 
    df.dropna(inplace= True)


# change all column "ร้อยละ" name
    cols = []
    count = 5
    for column in df.columns:
        if column == 'ร้อยละ':
            cols.append(f'{count}(%)')
            count-=1
            continue
        cols.append(column)
    df.columns = cols

    for i in range(2,15):
        df.iloc[:,i] = pd.to_numeric(df.iloc[:,i])
    df.set_index("ที่" , inplace= True)
# add  detail to data frame

    df["degree"] = head.iloc[3,1]
    df["semester"] = head.iloc[2,1]  
    df["subject"] = head.iloc[0,1]
    df["section"] = head.iloc[1,1]
    df["นักศึกษาที่จดทะเบียน"] = head.iloc[4,1]
    df["นักศึกษาที่ประเมิน"] = head.iloc[5,1]
    df["url"] = eval_url[page]

    dfs.append(df)
    
# concat all websites
result = pd.concat([dfs[i] for i in range(26)])

# separate year & semester 
result["year"] = ""
result["sem"] = ""

count = 0
for i in result["semester"]:
    yr_splt = i.split("/")
    result["year"][count] = yr_splt[1]
    result["sem"][count] = yr_splt[0]
    count += 1

# display

array(['https://sgulyano.github.io/eval/02226005.html',
       'https://sgulyano.github.io/eval/08846113.html',
       'https://sgulyano.github.io/eval/19689783.html',
       'https://sgulyano.github.io/eval/24363682.html',
       'https://sgulyano.github.io/eval/24674969.html',
       'https://sgulyano.github.io/eval/26742703.html',
       'https://sgulyano.github.io/eval/33472214.html',
       'https://sgulyano.github.io/eval/33561138.html',
       'https://sgulyano.github.io/eval/33565025.html',
       'https://sgulyano.github.io/eval/41557226.html',
       'https://sgulyano.github.io/eval/43951333.html',
       'https://sgulyano.github.io/eval/44868136.html',
       'https://sgulyano.github.io/eval/45435188.html',
       'https://sgulyano.github.io/eval/48033334.html',
       'https://sgulyano.github.io/eval/51435303.html',
       'https://sgulyano.github.io/eval/53231654.html',
       'https://sgulyano.github.io/eval/60189297.html',
       'https://sgulyano.github.io/eval/61116032

In [7]:
result["n"] = ""
for i in range(len(result["นักศึกษาที่ประเมิน"])):
    result["n"][i] = result["นักศึกษาที่ประเมิน"][i].split(" ")[0]
result["n"] = pd.to_numeric(result["n"])

# Situation Separate

* If question contain "covid-19" then situation = "covid"
* then else situaion is "normal"

In [8]:
# Build  key
result["class"] = result["semester"] + "|" + result["subject"] + "|" + result["section"]
covid = result[result["ข้อคำถาม"].str.contains("COVID-19")]
covid["situation"] = "covid"
covid = covid.filter(['class', 'situation'])
covid.drop_duplicates(inplace= True)

df = pd.merge(result,covid,on="class" , how="outer")
df["situation"].fillna("normal" , inplace=True)
df.rename(columns={"X-Bar" : "x_bar"},inplace= True)

#display
df

,ข้อคำถาม,5,5(%),4,4(%),3,3(%),2,2(%),1,...,subject,section,นักศึกษาที่จดทะเบียน,นักศึกษาที่ประเมิน,url,year,sem,n,class,situation
0,สอนอย่างเป็นระบบ,4,66.67,1,16.67,0,0.00,1,16.67,0,...,FGL497,231863,43,6 (13.95%),https://sgulyano.github.io/eval/02226005.html,2562,1,6,1/2562|FGL497|231863,normal
1,สอนให้คิด วิเคราะห์ วิจารณ์,4,66.67,0,0.00,1,16.67,1,16.67,0,...,FGL497,231863,43,6 (13.95%),https://sgulyano.github.io/eval/02226005.html,2562,1,6,1/2562|FGL497|231863,normal
2,กระตุ้นบรรยากาศการเรียนการสอน มีวิธีสอนที่น่าส...,2,33.33,1,16.67,1,16.67,1,16.67,1,...,FGL497,231863,43,6 (13.95%),https://sgulyano.github.io/eval/02226005.html,2562,1,6,1/2562|FGL497|231863,normal
3,เปิดโอกาสให้แสดงความคิดเห็นและแลกเปลี่ยนประสบก...,3,50.00,2,33.33,0,0.00,0,0.00,1,...,FGL497,231863,43,6 (13.95%),https://sgulyano.github.io/eval/02226005.html,2562,1,6,1/2562|FGL497|231863,normal
4,มีการให้นำเสนองาน/มีการวัดผลโดยวิธีอื่นๆที่สะท...,3,50.00,2,33.33,1,16.67,0,0.00,0,...,FGL497,231863,43,6 (13.95%),https://sgulyano.github.io/eval/02226005.html,2562,1,6,1/2562|FGL497|231863,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,ผู้สอนแทรกเนื้อหาเกี่ยวกับคุณธรรมจริยธรรมในการ...,15,51.72,9,31.03,3,10.34,2,6.90,0,...,NYE553,231863,43,29 (67.44%),https://sgulyano.github.io/eval/96905621.html,2563,1,29,1/2563|NYE553|231863,normal
320,โสตทัศนูปกรณ์ช่วยสอนในห้องเรียน เช่น ไมโครโฟน ...,14,48.28,8,27.59,5,17.24,1,3.45,1,...,NYE553,231863,43,29 (67.44%),https://sgulyano.github.io/eval/96905621.html,2563,1,29,1/2563|NYE553|231863,normal
321,สภาพของห้องเรียนหรือห้องปฏิบัติการ เช่น แสง อุ...,13,44.83,11,37.93,4,13.79,0,0.00,1,...,NYE553,231863,43,29 (67.44%),https://sgulyano.github.io/eval/96905621.html,2563,1,29,1/2563|NYE553|231863,normal
322,จำนวนอุปกรณ์ในการเรียนการสอนเพียงพอ และเหมาะสม...,15,51.72,6,20.69,5,17.24,0,0.00,3,...,NYE553,231863,43,29 (67.44%),https://sgulyano.github.io/eval/96905621.html,2563,1,29,1/2563|NYE553|231863,normal


In [11]:
df.drop(columns=['url','class'] , inplace=True)

In [12]:
# Save df to excel
df.to_excel("data.xlsx" , index=False)